In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import torch
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import os
import torchvision.transforms as transforms
from torchvision import datasets
from torchvision.transforms import ToTensor
from torchvision.transforms import Compose
from torchvision.transforms import Resize
import torch.optim as optim
from torch.utils.data import DataLoader , TensorDataset
from torch import Tensor
#!pip install einops
!pip install vit-pytorch
from torchvision.models import resnet18
from vit_pytorch.vit_for_small_dataset import ViT

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 43.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 52.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 75.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [3]:
from numpy import load
from sklearn.model_selection import train_test_split


images = load('/content/drive/My Drive/SYDE675_Leaf_Disease_Classification/KL_data/x_train.npy')
labels = load('/content/drive/My Drive/SYDE675_Leaf_Disease_Classification/KL_data/y_train.npy')

######### KL colab
# images = load('/content/drive/MyDrive/Colab_data/data_256/x_train.npy')
# labels = load('/content/drive/MyDrive/Colab_data/data_256/y_train.npy')

# images = load('x_train.npy')
# labels = load('y_train.npy')



x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)
x_train = x_train.reshape((-1,3,256,256))
x_test = x_test.reshape((-1,3,256,256))

In [4]:

print(np.shape(x_train))

(2880, 3, 256, 256)


In [5]:
torch.cuda.is_available()

True

In [6]:



image_size = 256

# Define transformations
train_transform = transforms.Compose([
    transforms.ToPILImage(),
#     transforms.RandomHorizontalFlip(),
#     transforms.RandomResizedCrop((256, 256), scale=(0.7, 1.0), ratio=(0.9, 1.1)),  # Random crop and resize
#     transforms.RandomRotation(degrees=(-10, 10), fill=(0,0,0)),  # Random rotation with black fill
#     transforms.ColorJitter(brightness=0.1, contrast=0.5, saturation=0.1),  # Random color adjustments
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

val_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

test_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])



# Apply transformations to custom datasets
x_train_tensor = torch.tensor(x_train, dtype=torch.float32)
x_test_tensor = torch.tensor(x_test, dtype=torch.float32)
x_train_processed = torch.stack([train_transform(image) for image in x_train_tensor])
x_test_processed = torch.stack([val_transform(image) for image in x_test_tensor])


# Create DataLoader objects
batch_size = 64

dataset_train = TensorDataset( x_train_processed, Tensor(y_train).long() )
dataset_test = TensorDataset( x_test_processed, Tensor(y_test).long())

train_loader = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset_test, batch_size=batch_size, shuffle=True)




In [7]:
# Initialize model, loss, and optimizer
num_epochs = 300

train_loss_rec = np.zeros(num_epochs)
train_acc_rec =np.zeros(num_epochs)
test_acc_rec = np.zeros(num_epochs)
# model  = MobileViT(
#     image_size = (256, 256),
#     dims = [96, 120, 144],
#     channels = [16, 32, 48, 48, 64, 64, 80, 80, 96, 96, 384],
#     num_classes = 2).to(device)
model = ViT(
    image_size = 256,
    patch_size = 16,
    num_classes = 2,
    dim = 1024,
    depth = 6,
    heads = 16,
    mlp_dim = 2048,
    dropout = 0.1,
    emb_dropout = 0.1).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.RAdam(model.parameters(), lr=1e-3, weight_decay=2e-2)

# Training loop
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        # Free up GPU memory
        del images, labels, outputs
        torch.cuda.empty_cache()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")
    train_loss_rec[epoch] = running_loss/len(train_loader)
    # Evaluation
    model.eval()
    train_correct = 0
    train_total = 0
    test_correct = 0
    test_total = 0
    with torch.no_grad():
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            train_total += labels.size(0)
            train_correct += (predicted == labels).sum().item()

            # Free up GPU memory
            del images, labels, outputs
            torch.cuda.empty_cache()


        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            test_total += labels.size(0)
            test_correct += (predicted == labels).sum().item()

            # Free up GPU memory
            del images, labels, outputs
            torch.cuda.empty_cache()

    print(f"train_Accuracy: {100 * train_correct / train_total}%")
    print(f"test_Accuracy: {100 * test_correct / test_total}%")
    train_acc_rec[epoch] =train_correct / train_total
    test_acc_rec[epoch] =test_correct / test_total

Epoch 1/300, Loss: 0.7261422965261671
train_Accuracy: 53.541666666666664%
test_Accuracy: 53.536754507628295%
Epoch 2/300, Loss: 0.719366619322035
train_Accuracy: 50.416666666666664%
test_Accuracy: 49.237170596393895%
Epoch 3/300, Loss: 0.7404756095674303
train_Accuracy: 50.38194444444444%
test_Accuracy: 49.237170596393895%
Epoch 4/300, Loss: 0.6925245417488946
train_Accuracy: 53.229166666666664%
test_Accuracy: 53.81414701803051%
Epoch 5/300, Loss: 0.6899994995858935
train_Accuracy: 56.354166666666664%
test_Accuracy: 54.92371705963939%
Epoch 6/300, Loss: 0.7022954185803731
train_Accuracy: 57.083333333333336%
test_Accuracy: 54.50762829403606%
Epoch 7/300, Loss: 0.6951371908187867
train_Accuracy: 57.361111111111114%
test_Accuracy: 57.004160887656035%
Epoch 8/300, Loss: 0.6924878809187147
train_Accuracy: 53.09027777777778%
test_Accuracy: 54.78502080443828%
Epoch 9/300, Loss: 0.696595287322998
train_Accuracy: 54.270833333333336%
test_Accuracy: 54.09153952843273%
Epoch 10/300, Loss: 0.683599

In [8]:
x_test = load('/content/drive/My Drive/SYDE675_Leaf_Disease_Classification/KL_data/x_test.npy')
y_test = load('/content/drive/My Drive/SYDE675_Leaf_Disease_Classification/KL_data/y_test.npy')

x_test = x_test.reshape((-1,3,256,256))
x_test_tensor = torch.tensor(x_test, dtype=torch.float32)
x_test_processed = torch.stack([test_transform(image) for image in x_test_tensor])

dataset_test = TensorDataset( x_test, Tensor(y_test).long())
test_loader = DataLoader(dataset_test, batch_size=batch_size, shuffle=True)

TypeError: 'int' object is not callable

In [ ]:
 # Evaluation
model.eval()

test_correct = 0
test_total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()

        # Free up GPU memory
        del images, labels, outputs
        torch.cuda.empty_cache()

test_acc =test_correct / test_total

In [ ]:
from numpy import save

save('/content/drive/My Drive/SYDE675_Leaf_Disease_Classification/KL_data/SV_train_loss_rec.npy', train_loss_rec)
save('/content/drive/My Drive/SYDE675_Leaf_Disease_Classification/KL_data/Sv_train_acc_rec.npy', train_acc_rec)
save('/content/drive/My Drive/SYDE675_Leaf_Disease_Classification/KL_data/Sv_val_acc_rec.npy', test_acc_rec)
save('/content/drive/My Drive/SYDE675_Leaf_Disease_Classification/KL_data/SV_test_acc.npy', test_acc)

# save('/content/drive/MyDrive/Colab_data/data_256/SV_train_loss_rec.npy', train_loss_rec)
# save('/content/drive/MyDrive/Colab_data/data_256/Sv_train_acc_rec.npy', train_acc_rec)
# save('/content/drive/MyDrive/Colab_data/data_256/Sv_val_acc_rec.npy', test_acc_rec)
# save('/content/drive/MyDrive/Colab_data/data_256/SV_test_acc.npy', test_acc)